# DEPENDÊNCIAS GERAIS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from google.colab import drive
from itertools import product
from sklearn.base import BaseEstimator, clone
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from xgboost import XGBRegressor

# FUNÇÕES

## Utilitário

In [ ]:
def dict_combine(dictionary: dict) -> list:
    """
    Gera todas as combinações possíveis entre os valores de um dicionário de listas,
    retornando uma lista de dicionários com cada combinação única.

    :param dictionary: Dicionário em que cada chave está associada a uma lista de valores possíveis.
                       Os valores devem ser iteráveis (como listas ou tuplas) e de mesmo comprimento.

    :return: Lista contendo dicionários. Cada dicionário representa uma combinação possível
             entre os valores das listas fornecidas no dicionário original. As chaves são mantidas,
             e os valores correspondem a uma das combinações do produto cartesiano.
    """
    # Passo 1: Obtemos as chaves do dicionário (ex: ["key1", "key2"])
    keys = list(dictionary.keys())

    # Passo 2: Obtemos os valores associados a cada chave (listas de possibilidades)
    # Exemplo: [["a", "b", "c"], ["d", "e", "f"]]
    values = list(dictionary.values())

    # Passo 3: Calculamos o produto cartesiano dessas listas de valores
    # Isso gera todas as combinações possíveis, como ("a", "d"), ("a", "e"), ..., ("c", "f")
    combinations = product(*values)  # Asterisco faz o desempacotamento dos elementos de values

    # Passo 4: Para cada combinação, associamos os valores às suas respectivas chaves
    # Isso é feito com a função zip, e transformamos o resultado em um dicionário
    # Resultado final: uma lista de dicionários, cada um representando uma combinação possível
    return [dict(zip(keys, combo)) for combo in combinations]

def dict_flatten(dictionary: dict, fields_to_flatten: list[str]) -> dict:
    """
    Desestrutura os campos selecionados de um dicionário, retornando apenas os atributos desejados.

    :param dictionary: Dicionário original que contém os campos a serem desestruturados.
    :param fields_to_flatten: Lista de nomes dos campos que devem ser desestruturados (flatten).

    :return: Novo dicionário contendo apenas os atributos dos campos selecionados, combinados em um nível.
    """
    dict_flat = dict()

    for field in fields_to_flatten:
        value = dictionary.get(field)
        if isinstance(value, dict):
            dict_flat.update(value)

    return dict_flat

def dict_to_flat_df(dictionary: dict, fields_to_flatten: list[str], key_name: str = "key") -> pd.DataFrame:
    """
    Converte um dicionário com estrutura aninhada em um DataFrame tabular, desestruturando
    apenas os campos especificados.

    :param dictionary: Dicionário em que cada chave representa um identificador único e cada valor é um dicionário com possíveis campos aninhados.
    :param fields_to_flatten: Lista de nomes de campos a serem desestruturados. Cada um deve
                               corresponder a uma chave cujo valor é um dicionário.
    :param key_name: Nome da coluna que armazenará os identificadores do dicionário original.
                     Padrão é "key".

    :return: DataFrame com uma coluna para os identificadores e colunas adicionais contendo os
             atributos resultantes da desestruturação dos campos selecionados.
    """
    rows = list()

    for identifier, nested_dict in dictionary.items():
        dict_flat = dict_flatten(nested_dict, fields_to_flatten)
        dict_flat[key_name] = identifier
        rows.append(dict_flat)


    df = pd.DataFrame(rows)
    df = df[[key_name] + [col for col in df.columns if col != key_name]]  # Coloca a chave primeiro
    return df

def key_part_build(attr, val, separator: str = "=") -> str:
    return separator.join([str(attr).upper(), str(val).lower()])

def key_build(key_parts: list[str], separator: str = "|") -> str:
    return separator.join(key_parts)

def key_build_from_params(params: dict[str]):
    parts = [key_part_build(k, v) for k, v in params.items()]
    return key_build(parts)

def series_combine(list_series: list[pd.Series], operations: list[-1, 1]):
    """
    Combina uma lista de séries aplicando multiplicações e divisões de acordo com a lista de operações.

    Args:
        list_series (list of pd.Series): Lista de séries a combinar.
        operations (list of int): Lista de 1 e -1, indicando multiplicação (1) ou divisão (-1).

    Returns:
        pd.Series: Série resultante da combinação.
    """
    if len(list_series) != len(operations):
        raise ValueError("'list_series' e 'operations' devem ter o mesmo tamanho.")

    # Inicializa com uma série de 1s do mesmo índice da primeira série
    result = pd.Series(1.0, index=list_series[0].index)

    for series, op in zip(list_series, operations):
        if op == 1:
            result *= series  # multiplica se op for 1
        elif op == -1:
            result /= series  # divide se op for -1
        else:
            raise ValueError("A lista de operações só pode conter 1 ou -1.")

    return result

def series_normalize(series: pd.Series, scaler, series_name: str) -> pd.Series:
    series_copy = series.copy()
    array = scaler.fit_transform(series_copy.values.reshape(-1, 1))
    return pd.Series(array.squeeze(), name=series_name)

## Exibição

In [ ]:
def df_show_head(df: pd.DataFrame, n: int = 5) -> None:
    display(df.head(n))
    print(f"Shape: {df.shape}")

## Validação

In [ ]:
def is_valid_col_selection(col_selection: dict) -> bool:
    return col_selection is not None

def is_valid_series(se: pd.Series) -> bool:
    return se is not None and (not se.empty) and se.notnull().all() and pd.api.types.is_numeric_dtype(se)

## Pré-Processamento

In [ ]:
def subs_outliers(df: pd.DataFrame, columns: list[str], view: bool = False) -> pd.DataFrame:
    """
    Substitui outliers pela média. Se view for True, exibe
    dois gráficos para cada coluna: antes e depois da mudança.
    """

    # Guarda resultado anterior
    df_before = df.copy()

    for column in columns:
        # Obtém quartis 1 e 3
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)

        # Obtém IQR (Intervalo Interquartílico)
        iqr = q3 - q1

        """
        Outlier se em (-inf, Q1 - 1.5 * IQR) ou (Q3 + 1.5 * IQR, +inf).
        Subsititui outliers por NA ('not available') segundo método IQR (InterQuartile Range).
        """
        df.loc[(df[column] < q1 - 1.5 * iqr) | (df[column] > q3 + 1.5 * iqr), column] = pd.NA

        # Substitui NAs pela média
        df[column] = df[column].fillna(df[column].mean())

    if view:
        for column in columns:
            # Exibe resultado inicial
            plt.boxplot(df_before[column])
            plt.title(f"ANTES: {column}")
            plt.show()

            # Exibe novo resultado
            plt.boxplot(df[column])
            plt.title(f"DEPOIS: {column}")
            plt.show()

    return df

## Validação Cruzada

In [ ]:
def cross_val(model: BaseEstimator,
              X: pd.DataFrame,
              y: pd.Series,
              folds: int = 5,
              metric: str = "r2",
              shuffle: bool = True,
              random_state: int = None) -> tuple[list, list]:
    """
    Realiza validação cruzada utilizando K-Fold para treinar e avaliar um modelo,
    com base em uma métrica de desempenho fornecida.

    O modelo é clonado para cada divisão dos dados, garantindo independência entre os folds.

    :param model: Estimador compatível com o Scikit-learn (deve implementar os métodos `fit` e `predict`).
    :param X: Conjunto de dados de entrada (features).
    :param y: Vetor alvo (target) correspondente às amostras em `X`.
    :param folds: Número de partições (folds) na validação cruzada. O padrão é 5.
    :param metric: Métrica de avaliação utilizada em cada fold.
                   Pode ser uma das opções: `"r2"`, `"mse"`, `"mae"`.
                   Se inválida, `"r2"` será usada como padrão.
    :param shuffle: Indica se os dados devem ser embaralhados antes da divisão em folds.
    :param random_state: Semente de aleatoriedade para reprodutibilidade. `None` implica aleatoriedade completa.

    :return: Tupla contendo:
             - lista de modelos treinados (um por fold)
             - lista de métricas correspondentes a cada fold
    """
    DICT_METRICS = {
        "r2":   r2_score,
        "mse":  mean_squared_error,
        "mae":  mean_absolute_error
    }
    models = list() # Salva modelos por fold
    metrics = list() # Salva resultados por fold
    metric_to_use = DICT_METRICS.get(metric, r2_score) # Define métrica a ser usada (padrão é coeficiente de determinação)

    # Define os índices que delimitam as partições
    kfolds = KFold(n_splits=folds, shuffle=shuffle, random_state=random_state)

    for train_index, test_index in kfolds.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        copy = clone(model) # Evita sobrescrição do modelo
        copy.fit(X_train, y_train)

        models.append(copy)
        metrics.append(metric_to_use(y_test, copy.predict(X_test))) # Calcula métrica

    return models, metrics

## Casos de pré-processamento

In [ ]:
# --- CONSTANTES SIMBÓLICAS ---
"""
Incluímos '_' ao início para evitar conflitos de nomes com o restante do código.
"""
# - Pré-processamento -
_COLUMN_REMOVAL = "col_rem"
_COLUMNS = "columns"
_NAME = "name"
_FEATURE_ENGINEERING = "feat_eng"
_OUTLIERS = "outliers"
_NORMALIZATION = "normalization"
_PCA = "pca"

PREFIXES = {
    _COLUMN_REMOVAL: "FS",
    _FEATURE_ENGINEERING: "FE",
    _OUTLIERS: "OUT",
    _NORMALIZATION: "NORM",
    _PCA: "PCA"
}

_ABSENT = "absent"
_TRUE = "true"
_FALSE = "false"

RADIXES = {
    _ABSENT: "na",
    _TRUE: "1",
    _FALSE: "0"
}

_X = "X"
_y = "y"
_PARAMS = "params"

# - Métricas -
_MAE = "mae"
_MSE = "mse"
_R2 = "r2"
DICT_METRICS = {
        _MAE: mean_absolute_error,
        _MSE: mean_squared_error,
        _R2:  r2_score,
}
_AVERAGE = "avg"
_BEST_MODEL = "best_model"
_ALGORITHM = "algorithm"
_CROSS_VAL = "cv"
_FOLD = "fold"
_TEST = "test"
_PRED = "y_pred"
_IMPORTANCE = "importance"

# FUNÇÕES
def apply_column_removal(df, X, y, col_selection, prefix, radixes):
    if is_valid_col_selection(col_selection):
        columns = col_selection[_COLUMNS]
        df = df.drop(columns=columns)
        key = key_part_build(prefix, col_selection[_NAME]) # Indica nome da seleção de colunas
    else:
        key = key_part_build(prefix, radixes[_ABSENT])
        columns = None

    return df, X, y, key, {_COLUMN_REMOVAL: columns}

def apply_feature_engineering(df, X, y, feat_eng, prefix, radixes):
    if is_valid_series(feat_eng):
        df = pd.concat([df, feat_eng], axis=1)
        key = key_part_build(prefix, feat_eng.name.lower) # Indica nome da característica
        feat_name = feat_eng.name
    else:
        key = key_part_build(prefix, radixes[_ABSENT])
        feat_name = None

    return df, X, y, key, {_FEATURE_ENGINEERING: feat_name}

def apply_outliers(df, X, y, out, prefix, radixes):
    if out:
        df = subs_outliers(df, df.columns, view=False)
        key = key_part_build(prefix, radixes[_TRUE])
    else:
        key = key_part_build(prefix, radixes[_FALSE])

    X = df.drop(columns=[y.name])
    y = df[y.name].squeeze() # Squeeze garante ser uma série

    return df, X, y, key, {_OUTLIERS: out}

apply_normalization_scaler = StandardScaler() # Guarda scaler para reuso
def apply_normalization(df, X, y, norm, prefix, radixes):
    if norm:
        X = pd.DataFrame(apply_normalization_scaler.fit_transform(X), columns=X.columns, index=X.index)
        key = key_part_build(prefix, radixes[_TRUE])
    else:
        key = key_part_build(prefix, radixes[_FALSE])

    return df, X, y, key, {_NORMALIZATION: norm}

def apply_pca(df, X, y, pca_n, prefix, radixes):
    if pca_n > 0: # Se for passado um número não positivo, entende-se que não há PCA
        key = key_part_build(prefix, pca_n)
        # Evita que componentes superem total de colunas, o que geraria redundância
        pca_n = min(len(X.columns), pca_n)

        pca_obj = PCA(n_components=pca_n)
        pca_arr = pca_obj.fit_transform(X)
        X_pca = pd.DataFrame(
            pca_arr,
            index=X.index,
            columns=[f"pc{i + 1}" for i in range(pca_n)]
        )
    else:
        key = key_part_build(prefix, radixes[_ABSENT])
        X_pca = X

    return df, X_pca, y, key, {_PCA: pca_n}

def get_preprocessing_cases(df: pd.DataFrame,
                            target: str,
                            random_state: int,
                            col_selections: list[dict],
                            features: list[pd.Series],
                            outliers: list[bool],
                            normalization: list[bool],
                            pcas: list[int]) -> dict:
    """
    Gera múltiplos cenários de pré-processamento combinando, seleção e engenharia de atributos,
    tratamento de outliers, normalização e redução de dimensionalidade (PCA).

    Cada cenário gera um conjunto `X`, `y` e um dicionário com parâmetros das transformações aplicadas.
    As chaves do dicionário final descrevem o pipeline aplicado via prefixos e sufixos codificados.

    :param df: DataFrame original contendo os dados brutos.
    :param target: Nome da coluna alvo (variável dependente) no DataFrame.
    :param random_state: Semente para controle de aleatoriedade (reprodutibilidade).
    :param col_selection: Lista de dicionários com atributos "name" (representação do cenário) e "columns" (lista de colunas a serem removidas).
    :param features: Lista de séries representando atributos derivados para engenharia de características.
                     Pode conter séries válidas ou `None` para ausência de feature engineering.
    :param outliers: Lista indicando se o tratamento de outliers deve ser aplicado.
    :param normalization: Lista indicando se a normalização (z-score) deve ser aplicada.
    :param pcas: Lista com números de componentes principais a serem testados via PCA.

    :return: Dicionário cujas chaves são strings descritivas do pipeline (ex: 'FEna_OUT1_NORM0_PCA2'), e
             os valores são dicionários contendo:
             - `_X`: DataFrame com os atributos processados.
             - `_y`: Série da variável alvo.
             - `_PARAMS`: Parâmetros que descrevem as transformações aplicadas no cenário.
    """
    dict_cases = dict() # Armazena os casos de experimentação
    df_original = df.copy() # Salva estado original do DataFrame

    np.random.seed(random_state) # Fixa semente aleatória

    for select, feat, out, norm, pca in product(col_selections, features, outliers, normalization, pcas):
        # Etapas de pré-processamento
        steps = [
            (apply_column_removal, select, PREFIXES[_COLUMN_REMOVAL]),
            (apply_feature_engineering, feat, PREFIXES[_FEATURE_ENGINEERING]),
            (apply_outliers, out, PREFIXES[_OUTLIERS]),
            (apply_normalization, norm, PREFIXES[_NORMALIZATION]),
            (apply_pca, pca, PREFIXES[_PCA]),
        ]

        key_parts = list()          # Armazena partes da chave que descreve o caso
        dict_params = dict()        # Armazena os parâmetros que compõem o caso
        df = df_original.copy()     # Copia DataFrame original para evitar sobrescrição
        X = df.drop(columns=[target])
        y = df[target]

        # Executa etapas
        for function, param, prefix in steps:
            df, X, y, key, update = function(df, X, y, param, prefix, RADIXES)
            key_parts.append(key)
            dict_params.update(update)

        key = key_build(key_parts)
        dict_cases[key] = {_X: X, _y: y, _PARAMS: dict_params}

    return dict_cases

def get_best_model(metric, models, scores):
    DICT_CRITERIA = {
        _MAE: np.argmin,
        _MSE: np.argmin,
        _R2: np.argmax
    }
    best_index = DICT_CRITERIA[metric](scores)
    best_index = best_index[0] if isinstance(best_index, np.ndarray) else best_index
    return models[best_index]

def get_model_cases(model: BaseEstimator,
                    pre_processing_cases: dict,
                    params: dict,
                    folds: list[int],
                    metrics: list[str],
                    test_size: float,
                    random_state: int,
                    log: bool = True) -> dict:
    """
    Executa experimentos completos de treinamento e avaliação de um modelo para múltiplos
    cenários de pré-processamento e combinações de hiperparâmetros.

    Para cada combinação de cenário de dados, configuração de hiperparâmetros e número de folds,
    realiza validação cruzada, seleciona o melhor modelo segundo uma métrica especificada e avalia
    seu desempenho em um conjunto de teste.

    :param model: Estimador compatível com Scikit-learn que será ajustado aos dados.
    :param pre_processing_cases: Dicionário de cenários de pré-processamento, em que cada chave representa
                                 um pipeline aplicado e os valores contêm os dados transformados (`X`, `y`).
    :param params: Dicionário de listas de valores a serem combinados como hiperparâmetros do modelo.
    :param folds: Lista com os diferentes valores de `k` a serem usados na validação cruzada (K-Fold).
    :param metrics: Lista de métricas a serem utilizadas para avaliação (`"r2"`, `"mae"`, `"mse"`).
    :param test_size: Proporção dos dados reservada para teste na separação treino-teste.
    :param random_state: Semente para garantir reprodutibilidade na separação dos dados e na validação cruzada.
    :param log: Indica se as mensagens de progresso devem ser exibidas.

    :return: Dicionário em que cada chave representa um cenário único (pipeline + hiperparâmetros) e
             os valores contêm:
                - `_PARAMS`: dicionário de hiperparâmetros usados no modelo
                - `_BEST_MODEL`: dicionário com os melhores modelos por métrica (escolhidos com base na V.C.)
                - `_CROSS_VAL`: dicionário com as métricas por fold e médias das validações cruzadas
                - `_TEST`: dicionário com as métricas obtidas no conjunto de teste por modelo selecionado
                - `_PRED`: dicionário com as predições (`y_pred`) dos melhores modelos no conjunto de teste,
                permitindo reconstruir resíduos
    """
    count = 0
    dict_cases = dict() # Armazena os casos de experimentação
    model_name = model.__class__.__name__

    # Percorre cenários de pré-processamento
    for base_key, pre_proc_params in pre_processing_cases.items():
        try:
            # Inicio da nova chave formado pela chave do pré-processamento e nome do modelo
            name_key_part = key_part_build(_ALGORITHM, model_name)
            base_key = key_build([base_key, name_key_part])

            # Gera conjuntos de treino e de teste
            X = pre_proc_params[_X]
            y = pre_proc_params[_y]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

            # Gera cenários com base no número de folds e nos parâmetros do modelo
            for model_params, fold_num in product(dict_combine(params), folds):
                count += 1
                # dict_best_models = dict() # Armazena melhores modelos por métrica
                dict_scores_cross_val = dict() # Armazena pontuações médias da V.C. por métrica
                dict_scores_test = dict() # Armazena pontuações do teste por métrica
                # dict_y_pred_test = dict() # Armazena predições do teste por métrica

                # Gera nova chave
                key_model_params = key_build_from_params(model_params)
                key_cv_part = key_part_build(_CROSS_VAL, fold_num)
                key_new = key_build([base_key, key_cv_part, key_model_params])

                model_case = clone(model)
                model_case.set_params(**model_params)

                for metric in metrics:
                    models_cross_val, scores_cross_val = cross_val(
                                        model_case,
                                        X_train,
                                        y_train,
                                        folds=fold_num,
                                        metric=metric,
                                        shuffle=True,
                                        random_state=random_state
                    )

                    # Guarda métricas por fold
                    for fold, score in zip(range(fold_num), scores_cross_val):
                        key_cross_val_folds = key_build([metric, _CROSS_VAL, _FOLD, str(fold)], separator="_")
                        dict_scores_cross_val[key_cross_val_folds] = score

                    # Computa métrica média
                    key_cross_val_avg = key_build([metric, _CROSS_VAL, _AVERAGE], separator="_")
                    dict_scores_cross_val[key_cross_val_avg] = np.mean(scores_cross_val)

                    # Escolhe melhor modelo
                    # key_cross_val_best_model = key_build([metric, _CROSS_VAL, _BEST_MODEL])
                    best_model = get_best_model(metric, models_cross_val, scores_cross_val)
                    # dict_best_models[key_cross_val_best_model] = best_model

                    # Computa métricas do melhor modelo
                    y_pred = best_model.predict(X_test)
                    for internal_metric in metrics:
                        key_best_model_test = key_build([metric, _BEST_MODEL, _TEST, internal_metric], separator="_")
                        dict_scores_test[key_best_model_test] = DICT_METRICS[internal_metric](y_test, y_pred)

                    # Armazena predições para reconstrução futura dos resíduos
                    # key_y_pred_test = key_build([metric, _BEST_MODEL, _TEST, _PRED])
                    # dict_y_pred_test[key_y_pred_test] = y_pred

                # Gera dicionário de parâmetros
                combined_params = dict(pre_proc_params[_PARAMS]) # Garante que é um dicionário
                combined_params.update(model_params)
                combined_params[_ALGORITHM] = model_name

                # Extrai feature importance
                if hasattr(best_model, "feature_importances_"):
                    importances = pd.Series(best_model.feature_importances_, index=X.columns)

                elif hasattr(best_model, "get_booster"):
                    raw_importances = best_model.get_booster().get_score(importance_type="gain")
                    importances = pd.Series(raw_importances)

                    # Ajusta para X.columns se as chaves forem 'f0', 'f1', ...
                    if all(k.startswith("f") for k in importances.index):
                        importances.index = [X.columns[int(k[1:])] for k in importances.index]
                else:
                    importances = None  # Ou raise Exception se quiser abortar

                dict_cases[key_new] = {
                    _PARAMS: {**combined_params},
                    _IMPORTANCE: {_IMPORTANCE: importances},
                    _CROSS_VAL: dict_scores_cross_val,
                    _TEST: dict_scores_test,
                    # _PRED: dict_y_pred_test,
                }

                if log:
                    print(f"{count:4d} - Caso {key_new} finalizado.")

        except Exception as e:
            print(f"Erro ao processar caso {key_new}: {e}")

    return dict_cases

# SALVAMENTO

In [ ]:
def dict_deserialize(path: str) -> dict:
    """
    Desserializa (carrega) um dicionário Python a partir de um arquivo .pkl.

    :param path: Caminho completo do arquivo a ser carregado.
                 Deve terminar com a extensão '.pkl' (ex: 'dict_cases.pkl').
    :return: Dicionário Python carregado a partir do arquivo.
    """
    return joblib.load(path)

def dict_serialize(dictionary: dict, path: str) -> None:
    """
    Serializa (salva) um dicionário Python em um arquivo no formato .pkl.

    :param dictionary: Dicionário a ser serializado.
    :param path: Caminho completo do arquivo de destino.
                 Deve terminar com a extensão '.pkl' (ex: 'dict_cases.pkl').
    """
    joblib.dump(dictionary, path)

def generate_csv(dict_cases: dict, path: str) -> None:
    FLATTEN = [_PARAMS, _CROSS_VAL, _TEST, _IMPORTANCE] # Campos a serem passados para o primeiro nível
    KEY_NAME = "case" # Nome do campo que armazenará o identificador dos casos

    """
    Achata campos que são dicionários, deixando todas as informações no primeiro
    nível. Em seguida, transforma em um DataFrame.
    """
    df_final = dict_to_flat_df(dict_cases, FLATTEN, KEY_NAME)
    df_final.to_csv(path, index=False)

def generate_sheet(dict_cases: dict, sheet_name: str, path: str) -> None:
    """
    Gera uma planilha Excel (.xlsx) contendo os dados tabulares dos casos modelados.

    :param dict_cases: Dicionário onde cada chave identifica um caso e o valor é um
                       dicionário com os dados do modelo, parâmetros e métricas.
    :param sheet_name: Nome da aba (sheet) que será criada no arquivo Excel.
    :param path: Caminho completo do arquivo .xlsx que será criado.
                 Deve terminar com a extensão '.xlsx' (ex: 'resultados.xlsx').

    A função realiza o seguinte:
    - Achata os campos definidos em FLATTEN (campos dicionário), elevando-os ao primeiro nível.
    - Concatena os dados em um único DataFrame.
    - Escreve esse DataFrame em uma planilha Excel na aba especificada.
    """
    FLATTEN = [_PARAMS, _CROSS_VAL, _TEST, _IMPORTANCE] # Campos a serem passados para o primeiro nível
    KEY_NAME = "case" # Nome do campo que armazenará o identificador dos casos

    """
    Achata campos que são dicionários, deixando todas as informações no primeiro
    nível. Em seguida, transforma em um DataFrame.
    """
    df_final = dict_to_flat_df(dict_cases, FLATTEN, KEY_NAME)

    with pd.ExcelWriter(path, engine="openpyxl") as writer:
        df_final.to_excel(writer, sheet_name=sheet_name, index=False)

# PLANO DE EXPERIMENTAÇÃO

## SETUP DO AMBIENTE

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


## OBTENÇÃO DO DATASET

In [ ]:
PATH = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/_all/all_merged.csv"
df = pd.read_csv(PATH)
df_show_head(df)

,_ano,_estado,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,cli_veloc_vento_max,cli_veloc_vento_med,que_area_queimada,que_focos_qtd
0,2008,AC,7,2.627698e+07,986.843612,28.142731,18.914978,59.555066,95.0,54.432558,29.0,5.1,2.152915,4957.0,165.0
1,2008,AC,9,2.627698e+07,991.705941,24.446194,19.467987,75.811881,97.0,72.496700,25.0,1.0,0.210504,46073.0,2947.0
2,2008,AC,10,2.627698e+07,990.328360,25.229298,21.617473,81.870968,96.0,78.720430,29.0,1.0,0.204959,30355.0,856.0
3,2008,AC,11,2.627698e+07,988.610987,25.195410,22.624478,86.905424,96.0,84.048679,42.0,1.0,0.186970,2082.0,63.0
4,2008,AC,12,2.627698e+07,988.692608,24.898790,22.727554,88.529570,96.0,86.116935,53.0,1.0,0.179442,127.0,4.0


Shape: (1025, 15)


## CODIFICAÇÃO

In [ ]:
# Codificação OneHot simplificada
TO_ENCODE = ["_estado"]
df_encoded = pd.get_dummies(df, columns=TO_ENCODE, dtype="Int32")

df_show_head(df_encoded)

,_ano,_mes,car_c02_emitido,cli_pressao_atm_med,cli_temp_ar_med,cli_temp_orvalho_med,cli_umid_rel_med,cli_umid_rel_min_max,cli_umid_rel_min_med,cli_umid_rel_min_min,...,cli_veloc_vento_med,que_area_queimada,que_focos_qtd,_estado_AC,_estado_AM,_estado_AP,_estado_PA,_estado_RO,_estado_RR,_estado_TO
0,2008,7,2.627698e+07,986.843612,28.142731,18.914978,59.555066,95.0,54.432558,29.0,...,2.152915,4957.0,165.0,1,0,0,0,0,0,0
1,2008,9,2.627698e+07,991.705941,24.446194,19.467987,75.811881,97.0,72.496700,25.0,...,0.210504,46073.0,2947.0,1,0,0,0,0,0,0
2,2008,10,2.627698e+07,990.328360,25.229298,21.617473,81.870968,96.0,78.720430,29.0,...,0.204959,30355.0,856.0,1,0,0,0,0,0,0
3,2008,11,2.627698e+07,988.610987,25.195410,22.624478,86.905424,96.0,84.048679,42.0,...,0.186970,2082.0,63.0,1,0,0,0,0,0,0
4,2008,12,2.627698e+07,988.692608,24.898790,22.727554,88.529570,96.0,86.116935,53.0,...,0.179442,127.0,4.0,1,0,0,0,0,0,0


Shape: (1025, 21)


## SELEÇÃO DE CARACTERÍSTICAS

In [ ]:
# Remoção dos estados
scn_stateless = {
    "name": "stateless",
    "columns": [col for col in df_encoded.columns if col.startswith("_estado")]
}

## SETUP

In [ ]:
# Constantes simbólicas
PRE = "pre_processing"              # Pré-processamento
RF = "random_forest"                # Random Forest
XGB = "xgboost"                     # XGBoost
MLR = "multiple_linear_regression"  # Regressão Linear Múltipla
ALL = "all"                         # Todos os modelos considerados simultaneamente

# Dicionários para armazenar os resultados
dict_cases = dict() # Armazena dicionários de casos
dict_df_cases = dict() # Armazena DataFrames de casos

# PARÂMETROS GERAIS
RANDOM_STATE = 42
TARGET = "que_area_queimada"
CONFIG = {
    "folds": [2, 3, 5],
    "metrics": [_MAE, _MSE, _R2],
    "test_size": 0.30,
    "random_state": RANDOM_STATE
}

# SALVAMENTO
PATH_OUT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/src/03_experiment_plan/plans"

## CASOS

### CASOS DE PRÉ-PROCESSAMENTO

In [ ]:
# Parâmetros auxiliares (mude se desejar)
PRE_PARAMS = {
    "df": df_encoded,
    "target": TARGET,
    "random_state": RANDOM_STATE,
    "col_selections": [None, scn_stateless],
    "features": [None],
    "outliers": [False, True],
    "normalization": [False, True],
    "pcas": [0, 5, 10]
}

# Obtém casos de pré-processamento
dict_cases[PRE] = get_preprocessing_cases(**PRE_PARAMS)
dict_df_cases[PRE] = pd.DataFrame(data=dict_cases[PRE]).T
df_show_head(dict_df_cases[PRE])

# Salva arquivos
path = f"{PATH_OUT}/{PRE}/{PRE}"
generate_csv(dict_cases[PRE], f"{path}.csv")
generate_sheet(dict_cases[PRE], PRE, f"{path}.xlsx")

,X,y,params
FS=na|FE=na|OUT=0|NORM=0|PCA=na,_ano _mes car_c02_emitido cli_pressao...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
FS=na|FE=na|OUT=0|NORM=0|PCA=5,pc1 pc2 pc3 ...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
FS=na|FE=na|OUT=0|NORM=0|PCA=10,pc1 pc2 pc3 ...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
FS=na|FE=na|OUT=0|NORM=1|PCA=na,_ano _mes car_c02_emitido cli...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."
FS=na|FE=na|OUT=0|NORM=1|PCA=5,pc1 pc2 pc3 pc4 ...,0 4957.0 1 46073.0 2 303...,"{'col_rem': None, 'feat_eng': None, 'outliers'..."


Shape: (24, 3)


### Regressão Linear Múltipla

In [ ]:
# Parâmetros auxiliares (mude se desejar)
MLR_PARAMS = {
    "positive":         [False],
    "fit_intercept":    [True]
}

dict_cases[MLR] = get_model_cases(
    model=LinearRegression(),
    params=MLR_PARAMS,
    pre_processing_cases=dict_cases[PRE],
    **CONFIG,
)
dict_df_cases[MLR] = pd.DataFrame(data=dict_cases[MLR]).T

df_show_head(dict_df_cases[MLR])

# Salva arquivos
path = f"{PATH_OUT}/{MLR}/{MLR}"
generate_csv(dict_cases[MLR], f"{path}.csv")
generate_sheet(dict_cases[MLR], MLR, f"{path}.xlsx")

   1 - Caso FS=na|FE=na|OUT=0|NORM=0|PCA=na|ALGORITHM=linearregression|CV=2|POSITIVE=false|FIT_INTERCEPT=true finalizado.
   2 - Caso FS=na|FE=na|OUT=0|NORM=0|PCA=na|ALGORITHM=linearregression|CV=3|POSITIVE=false|FIT_INTERCEPT=true finalizado.
   3 - Caso FS=na|FE=na|OUT=0|NORM=0|PCA=na|ALGORITHM=linearregression|CV=5|POSITIVE=false|FIT_INTERCEPT=true finalizado.
   4 - Caso FS=na|FE=na|OUT=0|NORM=0|PCA=5|ALGORITHM=linearregression|CV=2|POSITIVE=false|FIT_INTERCEPT=true finalizado.
   5 - Caso FS=na|FE=na|OUT=0|NORM=0|PCA=5|ALGORITHM=linearregression|CV=3|POSITIVE=false|FIT_INTERCEPT=true finalizado.
   6 - Caso FS=na|FE=na|OUT=0|NORM=0|PCA=5|ALGORITHM=linearregression|CV=5|POSITIVE=false|FIT_INTERCEPT=true finalizado.
   7 - Caso FS=na|FE=na|OUT=0|NORM=0|PCA=10|ALGORITHM=linearregression|CV=2|POSITIVE=false|FIT_INTERCEPT=true finalizado.
   8 - Caso FS=na|FE=na|OUT=0|NORM=0|PCA=10|ALGORITHM=linearregression|CV=3|POSITIVE=false|FIT_INTERCEPT=true finalizado.
   9 - Caso FS=na|FE=na|OUT

,params,importance,cv,test
FS=na|FE=na|OUT=0|NORM=0|PCA=na|ALGORITHM=linearregression|CV=2|POSITIVE=false|FIT_INTERCEPT=true,"{'col_rem': None, 'feat_eng': None, 'outliers'...",{'importance': None},"{'mae_cv_fold_0': 109547.4452057426, 'mae_cv_f...",{'mae_best_model_test_mae': 111221.01379464717...
FS=na|FE=na|OUT=0|NORM=0|PCA=na|ALGORITHM=linearregression|CV=3|POSITIVE=false|FIT_INTERCEPT=true,"{'col_rem': None, 'feat_eng': None, 'outliers'...",{'importance': None},"{'mae_cv_fold_0': 99754.59953279266, 'mae_cv_f...",{'mae_best_model_test_mae': 101671.82995565492...
FS=na|FE=na|OUT=0|NORM=0|PCA=na|ALGORITHM=linearregression|CV=5|POSITIVE=false|FIT_INTERCEPT=true,"{'col_rem': None, 'feat_eng': None, 'outliers'...",{'importance': None},"{'mae_cv_fold_0': 93409.29477602351, 'mae_cv_f...","{'mae_best_model_test_mae': 99788.4271801213, ..."
FS=na|FE=na|OUT=0|NORM=0|PCA=5|ALGORITHM=linearregression|CV=2|POSITIVE=false|FIT_INTERCEPT=true,"{'col_rem': None, 'feat_eng': None, 'outliers'...",{'importance': None},"{'mae_cv_fold_0': 105592.0147714832, 'mae_cv_f...",{'mae_best_model_test_mae': 111217.21858230318...
FS=na|FE=na|OUT=0|NORM=0|PCA=5|ALGORITHM=linearregression|CV=3|POSITIVE=false|FIT_INTERCEPT=true,"{'col_rem': None, 'feat_eng': None, 'outliers'...",{'importance': None},"{'mae_cv_fold_0': 105139.23445621124, 'mae_cv_...",{'mae_best_model_test_mae': 103522.68265099762...


Shape: (72, 4)


### Random Forest

In [ ]:
# Parâmetros auxiliares (mude se desejar)
RF_EST = 100
RF_DEPTH = 6

RF_PARAMS = PARAMS = {
    "n_estimators": [RF_EST],
    "max_depth":    [RF_DEPTH],
    "random_state": [RANDOM_STATE]
}

dict_cases[RF] = get_model_cases(
    model=RandomForestRegressor(),
    params=RF_PARAMS,
    pre_processing_cases=dict_cases[PRE],
    **CONFIG
)
dict_df_cases[RF] = pd.DataFrame(data=dict_cases[RF]).T

# Salva arquivos
path = f"{PATH_OUT}/{RF}/{RF}_{RF_EST}_{RF_DEPTH}" # Versão parcial, para permitir paralelização
generate_csv(dict_cases[RF], f"{path}.csv")
generate_sheet(dict_cases[RF], RF, f"{path}.xlsx")

### XGBoost

In [ ]:
# Parâmetros auxiliares (mude se desejar)
XGB_EST = 100
XGB_DEPTH = 6

XGB_PARAMS = {
    "n_estimators":             [XGB_EST],
    "max_depth":                [XGB_DEPTH],
    "random_state":             [RANDOM_STATE]
}

dict_cases[XGB] = get_model_cases(
    model=XGBRegressor(),
    params=XGB_PARAMS,
    pre_processing_cases=dict_cases[PRE],
    **CONFIG
)
dict_df_cases[XGB] = pd.DataFrame(data=dict_cases[XGB]).T

# Salva arquivos
path = f"{PATH_OUT}/{XGB}/{XGB}_{XGB_EST}_{XGB_DEPTH}" # Versão parcial, para permitir paralelização
generate_csv(dict_cases[XGB], f"{path}.csv")
generate_sheet(dict_cases[XGB], XGB, f"{path}.xlsx")